In [ ]:
import os
os.environ.setdefault('JAVA_HOME', '/usr/lib/jvm/java-1.8.0-openjdk-amd64')
import networkx as nx
import community
import matplotlib.pyplot as plt
import statistics
import pickle
import pyspark.ml

from pyspark.sql import SparkSession
from neo4j import GraphDatabase, Driver

Importing graph

In [ ]:
def import_graph(path):
    print('importing graph from :', path)
    g = nx.read_gexf(path, node_type=None, relabel=True).to_undirected()
    return g

Verification

In [ ]:
def verify_graph(g):
    print('number of nodes : ' + str(len(g)))
    fig, ax = plt.subplots(figsize=(100, 70)) # set size
    nx.draw(g, with_labels=True)
    plt.show()

Graph cleaning

In [ ]:
def clean_graph(graph):
    print('cleaning graph')
    #Degree computation
    nodes_with_degrees = graph.degree
    #mean
    mean_deg = statistics.mean(l[1] for l in nodes_with_degrees)
    #std
    std_deg = statistics.stdev(l[1] for l in nodes_with_degrees)
    #threshold
    threshold = mean_deg + std_deg*std_deg/2 

    #Filtering extremely highly connected nodes
    nodes_to_remove = list(filter(lambda d : d[1] > threshold, nodes_with_degrees))
    n,d = zip(*nodes_to_remove)
    graph.remove_nodes_from(n)
    
    #Filtering isolated nodes
    nodes_with_degrees = graph.degree
    nodes_to_remove = list(filter(lambda d : d[1] == 0, nodes_with_degrees))
    n,d = zip(*nodes_to_remove)
    graph.remove_nodes_from(n)

    print('remaining nodes : ' + str(len(graph)))
    
    return graph

Keeping only largest connected component

In [ ]:
def largest_connected_component(graph):
    print('largest connected component')
    gcc = max(nx.connected_component_subgraphs(graph), key=len)
    print('remaining nodes : ' + str(len(gcc)))
    return gcc

Partitioning using Louvain

In [ ]:
def communities_louvain(graph):
    louvain_communities = community.best_partition(graph, resolution=1)
    louvain_communities_dict = {}
    for key, value in sorted(louvain_communities.items()):
        louvain_communities_dict.setdefault(value, []).append(key)

    print('detcted',len(louvain_communities_dict),'communities')
    
    return louvain_communities_dict

Partitioning using Leiden


https://pypi.org/project/leidenalg/
https://www.nature.com/articles/s41598-019-41695-z

categoriy of each partition

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=('',''))

helpers

In [ ]:
#get list of categories of a page
def get_categories(page_name):
    c = list()
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for record in tx.run("MATCH (p:Page)-[:BELONGS_TO]->(c:Category) "
                                 "WHERE p.title = {page_name} "
                                 "AND NOT exists((c)-[:BELONGS_TO]->(:Category {title: \'Hidden_categories\'})) "
                                 "RETURN c.title", 
                                 page_name = page_name ):
                #print(record["c.title"])
                c.append(record["c.title"])
    return c

#map each element to frequency in a list    
def count_frequency(my_list): 
      
    # Creating an empty dictionary  
    freq = {} 
    for items in my_list: 
        freq[items] = my_list.count(items)
    return freq

#iterate over pages dict partition
def part_category_fetch(key, dic):
    cat = []
    for title in dic[key]:
        cat += get_categories(title)
    print('done fetching')
    return cat

def fetcher(bpd):
    part_cat = {}
    
    for part in sorted(bpd):
        print(part)
        cat = part_category_fetch(part, bpd)
        #print(cat)
        part_cat.setdefault(part, cat)
    
    return part_cat

Fetch categories for each cluster

In [ ]:
def fetch_categories(bpd):
    part_cat_dict = fetcher(bpd)
    
    return part_cat_dict

In [ ]:
def count_all_frequencies(d):
    part_cat_dict_freq = {}
    for e in d:
        cat_map_freq = count_frequency(part_cat_dict[e])
        part_cat_dict_freq.setdefault(e, cat_map_freq)
    return part_cat_dict_freq
 
def find_max_freq(p):
    max_part_cat = {}
    for e in p:
        ls = list(p[e].keys())
        cat = ls[0]
        for x in ls:
            if p[e][cat] < p[e][x]:
                cat = x
        max_part_cat.setdefault(e, cat)
    return max_part_cat

In [ ]:
g = import_graph("graphs/peaks_graph_20190901_20190915.gexf")
verify_graph(g)
gg = largest_connected_component(clean_graph(g))
verify_graph(gg)
communities = communities_louvain(gg)
part_cat_dict = fetch_categories(communities)


In [ ]:
print(part_cat_dict[len(part_cat_dict)-1])

Compute frequency of category in each part and find max freq cat for each part

In [ ]:
maxx = find_max_freq(count_all_frequencies(part_cat_dict))
for e in sorted(maxx):
    print(e, maxx[e])

#######################################################################################################################

In [ ]:
spark = SparkSession.builder.appName('LDA').config("spark.master", "local[*]").config("spark.sql.warehouse.dir", "/home/ayman/warehouse").getOrCreate()

In [ ]:
' '.join(part_cat_dict[len(part_cat_dict) - 1])

In [ ]:
df_ = []
for p in part_cat_dict:
    df_.append((p, ' '.join(part_cat_dict[p]).replace('_', ' ').replace(',','').replace('\\\'', ' ').lower()))
        
for e in df_:
    print(e)

In [ ]:
partitionsData = spark.createDataFrame(df_, ['cluster', 'categories'])

In [ ]:
print(partitionsData)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql.types import ArrayType, StringType
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [ ]:
tokenizer = Tokenizer(inputCol="categories", outputCol="raw")
wordsDirtyData = tokenizer.transform(partitionsData)
remover = StopWordsRemover(inputCol="raw", outputCol="wordsU")
wordsDataUnstemmed = remover.transform(wordsDirtyData)
print(wordsDataUnstemmed)
lemmatizer = WordNetLemmatizer()
lemmatizer_udf = udf(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens], ArrayType(StringType()))
wordsData = wordsDataUnstemmed.withColumn("words", lemmatizer_udf("wordsU")).select('cluster', 'categories', 'raw', 'words')
print(wordsData)

In [ ]:
print(wordsData.select('raw').first())
print(wordsData.select('words').first())

In [ ]:
#hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(wordsData)
#print(featurizedData)

In [ ]:
countVectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
cvmodel = countVectorizer.fit(wordsData)
featurizedData = cvmodel.transform(wordsData)
print(featurizedData)

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
print(rescaledData)

In [ ]:
dataset = rescaledData.select('cluster', 'features')
print(dataset)
dataset.show(truncate=False)

In [ ]:
from pyspark.ml.clustering import LDA

In [ ]:
# Trains a LDA model.
lda = LDA(k=len(part_cat_dict), maxIter=100)
ldaModel = lda.fit(dataset)

In [ ]:
ll = ldaModel.logLikelihood(dataset)
lp = ldaModel.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = ldaModel.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics.show()

# Shows the result
transformed = ldaModel.transform(dataset)
transformed.show()

Verification for one cluster:
getting the full row, getting the topic index and the corresponding topic informations

In [ ]:
l = transformed.select('topicDistribution').first()[0]
print(transformed.first())
m = list(l).index(max(l))
print('\ntopic index is :',m)
print(topics.take(m+1)[m])

Printing the 5 most relevant words and their weights for each topic. 

In [ ]:
topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)
vocabList = cvmodel.vocabulary
tops = []
for i,t,w in topicIndices.collect():
    print('Topic %d:' % i)
    entry = []
    for j in range(len(t)):
        entry.append(vocabList[t[j]])
        #print('\t', vocabList[t[j]], w[j])
    print(entry)
    tops.append(entry)

https://stackoverflow.com/questions/44233862/visualizing-topics-with-spark-lda

Visualizing array of topics

In [ ]:
print(tops)

Assign topic words to each cluster using LDA output

In [ ]:
cluster_topicDist = sorted(transformed.select('cluster', 'topicDistribution').collect())
cluster_topicTerms = []
for e in cluster_topicDist:
    m = list(e[1]).index(max(e[1]))
    print(m)
    cluster_topicTerms.append(tops[m])

Create DF with cluster number, its pages, the LDAtopics and the MAXcategory

In [ ]:
df2_ = []
for k in sorted(communities.keys()):
    df2_.append((k, communities[k], cluster_topicTerms[k], maxx[k]))
    
partitionsData2 = spark.createDataFrame(df2_, ['cluster', 'page names', 'LDAtopics', 'MAXcategory'])
partitionsData2.show()

distance between words using how many letters to flip<br>
checking the connected clusters (are they similar ?)<br>
stemming<br>
display the title of the most viewed page (in each cluster)<br>
find central node in each cluster (node_centrality)<br>
visualization<br>